In [ ]:
# Burada ilk önce modeli yükleyeceğiz.
# O yüzden dosya dizinini modelin bulunduğu klasör olarak değiştiriyoruz.
%cd yolov7

In [ ]:
# Ardından modeli yüklüyoruz. 
# Burada önceden eğitilmiş olan ağırlıkları kullandım.
# Burada kullanacağımız ağrıkların adını yazıyoruz.
# böylece modeli tanımlamış oluyoruz.
# Bu arada Pytorch kurulu olmalı bilgisayrınızda ayrıca cuda hali kurulu olmalı.
# yoksa hata alırsınız.
from  hubconf import custom
model = custom(path_or_model='yolov7.pt')  # custom example
# model = create(name='yolov7', pretrained=True, channels=3, classes=80, autoshape=True)  # pretrained example

In [ ]:
# Burada bir dizini bir üst dizin olarak değiştiriyıruz.
# Böylece kod dosyasının bulunduğu ana dizine geldik
%cd ..

In [ ]:
# Bundan sonra ise kullanacağımız tracker objesini tanımlayacağız. 
# Burada deep sort kullanıyoruz. O yüzden o dizinin içine giriyoruz.





# Bu arada buradaki dosyalar Github'taki ana DeepSORT reposundan daha farklı
# O repodaki bazı kütüphanlerin fonksiyonları artık olmadığı için hata veriyor.
# O yüzden buradaki klasördeki bazı dosyalarda değişiklikler yaptım.
# Bu çok önemli bir bilgi değil. Fakat orjinal DeepSORT reposunu kullanırsanız hata alırsınız.
# O yüzden bu benim repodaki DeepSORT'u kullanmanız gerekyor.




%cd deep_sort

In [ ]:
# burada gerekli objeleri import ediyoruz.

from deep_sort import  nn_matching
from application_util import preprocessing
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools import generate_detections as gdet

In [ ]:
# burada tracker objesindeki bazı parametreleri ayarlıyoruz.
max_cosine_distance = 0.4
nn_budget = None
nms_max_overlap = 1.0
    
# Burada tracker objesini tanımlıyoruz. Bunu kullnarak her bir nesneye bir id atıyrouz.

model_filename = 'mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

In [ ]:
# Tekrardan ana repo'ya dönüyoruz. 

%cd ..

In [ ]:
# Kütüphaneleri yüklüyoruz.

import cv2
import time
import numpy as np

In [ ]:
# Eğer ayrı bir environment oluşturduysanız bunu çalıştırmanıza gerek olmayabilir.
# fakat ana envirionment'ı kullanıyırsanız bunu çalıştırmazsanız tracking kısmında hata alabilirsiniz.

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# burada etiketleri tanımlıyoruz.


labels=[ 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
         'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
         'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
         'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
         'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
         'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
         'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
         'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
         'hair drier', 'toothbrush' ]

Koda geçmeden önce bahsetmem gereken birkaç detay var. Burada sadece arabaların hızını ölçüyoruz. Kullanılan mantık ise şöyle. Videodaki youn her iki tarafında da kesikli çizgiler var. Bu çizgilerin birbirine uzaklığı 9.144 metre (Bunu o yolla ilgili bir yazıdan aldım). Arabaların bir çizgiden diğer çizgiye gidene kadar geçen süreyi bularak bu uzunluğa bölüp ortalama hızı elde ettim.

Fakat bunun birkaç sorunu var. Arabaların orta noktalarının koordinatlarını kullanadım. Fakat bu orta tam olarak bu çizgilerin olduğu kordinatlara denk gelmiyor. Çoğunlukla 4-5 piksel arası bir fark oluyor. O yüzden resimdeki çizgilerin olduğu yerlere çizgi çekmek yerine dikdörtgen bölgeler tanımladım. Bunların dikey genişliği 14 ve 15 piksel. Böylece bu arabalar bu bölgeye girdiği zaman giriş zamanları kaydediliyor. 

Fakat burada da şöyle bir soun var. Normalde benim belirdiğim kesikli çizgiler arası piksel uzaklığı yolun sağında 86 piksel solunda ise 74 piksel. Fakat biz bölge tanımladığımız için  arabaların iki bölgeye girdiği kısımlar arası uzaklık bu sayılardan daha farklı oluyor. Bu durumda gerçekteki uzakalıktan daha farklı bir uzaklık değeri için süre bulunuyor. Burada da şöyle bir çözüm var. Arabaların o bölgelere girdiği noktaların dikey kordinat değerlerini de bulup birbirinden çıkarıp ardından bir oran orantı yapılıyor. Örneğin yolun sağındaki 86 piksel 9.144 metre ise bizim bulduğumuz x piksel  (x*9.144)/86 metre eder.
Buradaki işlem sonucu çok ufak bir hata payı oluşuyor. Ayrıca bu mantık birbirlerine çok yakın olduğu içi yapılıyor. Yoksa ekranın çok ykısımlarında bu mantık işlemez. Oradaki 86 piksel uzunluk gerçketeki çok daha farklı bir uzaklığa denk geliyordur.


Bir diğer önemli kısım ise burada nesnenin tespiti gibi işlemler olduğu için geçen süre videodaki geçen süreden çok daha uzun. O yüzden bulduğumuz hızı 30/FPS değeri ile çarptım. Buradaki 30 orijinal videonun 30 FPS olmasından dolayı. 

Son olarak çıkış bölgeleri çok daha büyük. Bunun sebebi arabanın hızı araba bu bölgede iken gözücekek şekilde ayarlandı. Fakat bölgenin büyük olmasının hesaplamaya bir katkısı yok. Çünkü arabanın bölgeye ilk girdiği değerleri alıyoruz.

In [ ]:
# burada kullanacağımız videonun adını giriyoruz.
kamera= cv2.VideoCapture('trafik.mp4')
# Yazınını fontunu belirliyoruz.
font = cv2.FONT_HERSHEY_SIMPLEX
# FPS hesaplamak için tanımlamalar.
prev_frame_time = 0
new_frame_time = 0

# Yolun sağ tarafı için bölgeler tanımlıyoruz.
# Bu arabanın girdiği bölge
region1=np.array([(700,575),(1100,575),(1100,560),(700,560)])
region1 = region1.reshape((-1,1,2))



# Bu arabanın çıktığı bölge
# Ayrıca ekranda bu bölge kırmızı ile gösteriliyor.
region2=np.array([(700,489),(1000,489),(1000,400),(700,400)])
region2 = region2.reshape((-1,1,2))






# Yolun sol tarafı için bölgeler tanımlıyoruz.
# Bu arabanın girdiği bölge
region3=np.array([(320,466),(580,466),(580,480),(320,480)])
region3 = region3.reshape((-1,1,2))



# Bu arabanın çıktığı bölge
# Ayrıca ekranda bu bölge kırmızı ile gösteriliyor.
region4=np.array([(200,540), (550,540),(520,660),(70,660)])
region4 = region4.reshape((-1,1,2))



# Sağ taraftaki kesikli çizgiler arasındaki piksel farkı
uzunluk=86

# SOL taraftaki kesikli çizgiler arasındaki piksel farkı
uzunluk2=74
# Kesikli çizgiler arasındaki uzaklık 
distance=9.144




# Buradakiler yolun sağı için
#burada sözlük kulanıyoruz. Böyelece bu bilgileri her arabaya özel ekliyoruz.
# Yani bir her bir arabanın id'si ile kaydediliyor bu bilgiler 

# Giriş bölgesine giren arabaların girdiği zamanı buna ekliyoruz.
vehicles_entering = {}
# Giriş bölgesine giren arabaların dikey konumunu buna ekliyoruz.
vehicles_konum = {}
# Arabaların hızını bulduktan sonra hzılarını buna ekliyoruz.
vehicles_speed = {}
# Arabaların iki bölge arasında süre farkını bulduktan sonra süreleri buna ekliyoruz.
vehicles_elapsed_time = {}


http://localhost:8888/notebooks/jupiter%20files/speed%20detection/Speed.ipynb#
# Buradakiler yolun solu  için

vehicles_entering2 = {}
vehicles_konum2 = {}
vehicles_speed2 = {}
vehicles_elapsed_time2 = {}




count=0
while True:
    #FPS'i hesaplıyoruz. 
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time

    
    ret,kare=kamera.read()
    # resmi RGB'ye çeviridk.
    imgs=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB)
    # Burada giriş bölgeleri yorumda.
    # Yani ekrana çizdirmedim.
    #cv2.polylines(kare,[region1],True,(0,0,255),2)
    cv2.polylines(kare,[region2],True,(0,0,255),2)
    #cv2.polylines(kare,[region1],True,(0,0,255),2)
    cv2.polylines(kare,[region4],True,(0,0,255),2)
    
    # resmi modele verip sonuçları elde ettik.
    results = model(imgs) 
    
    bboxes=[]
    scores=[]
    names=[]
    # Buradaki nesne sayısı kadar dönüyor döngü
    for i in range(len(results.xyxy[0])):
        xmin,ymin,xmax,ymax,score,name=results.xyxy[0][i]
        xmin,ymin,xmax,ymax=int(xmin),int(ymin),int(xmax),int(ymax)
        x1,y1=xmin,ymin
        w1=xmax-xmin
        h1=ymax-ymin
       
        # Eğer nesnenin score'u 0.5'ten küçükse onu almıyoruz.
        if score <0.5:
            continue
        # Sadece arabaları almak için bunu yapıyoruz. Arabaların indisi 2 çünkü
        if name!=2:
            continue
        
        bboxes.append([x1,y1,w1,h1])
        scores.append(score)
        names.append(name)
  
        
    # Bulunan sonuçları tracking işlemi için uygun formata çeviriyoruz.    
    features = encoder(imgs, bboxes)
    detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(bboxes, scores, features)]


    tracker.predict()
    # ilk frame'e bir işle uygulamyoruz.
    # O yüzden böyle if bloğu var.
    if count==0:
        count+=1
        continue
    # Tracker'a sonuçları vererek tracking işlemi yapıyrouz. 
    tracker.update(detections)
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 

        # Burada nesnelerin konumunu elde ediyoruz. Tracking sonrası
        bbox = track.to_tlwh()
        x,y,w,h=bbox
        x,y,w,h=int(x),int(y),int(w),int(h)
        # arabanın ortasının koordinatlarını buluyoruz.
        cx=int(x+w/2)
        cy=int(y+h/2)
        # orta noktaların y koordinatını kaydediyoruz. 
        # bunları ileride  kullanacağız
        # Bu sağ kısım için 
        cyy=cy
        # Bu sol kısım için 
        cyy2=cy
        
        # Nesnelerin (arabların) id'sini blulup bunu isim adlı bir değişkene kaydediyoruz.
        isim=track.track_id
        
        
        
        
        
        
        # Buradaki işlemleri yolun sol kısmı için yapıyrouz. 
        
        # Burada arabanın sağ giriş bölgesine girip girmediğine bakıyoruz.
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        # Eğer sağ giriş bölgesine girmişse bu if bloğu True döner ve buradaki blogun içine girilir.
        if inside_region1>0:
            # arabanın sağ  giriş bölgesine girdiği zaman bu sözlüğe kaydediliyor
            vehicles_entering[isim] = time.time()
            # arabanın sağ  giriş bölgesine girdiği dikey konum bu sözlüğe kaydediliyor
            vehicles_konum[isim]=cyy
        
        
        # Eğer araba sağ giriş bölgesine girmişse bu sözlükte bu kısım True döner ve buradaki blogun içine girilir.
        if isim in vehicles_entering:
            # Burada sağ çıkış bölgesine girip girmediğine bakıyoruz. 
            inside_region2=cv2.pointPolygonTest(region2,(cx,cy),False)
            # Eğer sağ çıkış bölgesine girmişse bu if bloğu True döner ve buradaki blogun içine girilir.
            if inside_region2>0:
                # Çıkış bölgesine girdiği zaman giriş bölgesine girdiği zamandan çıkarılır ve arada geçen süre bulunur
                elapsed_time = time.time() - vehicles_entering[isim]
                # Çıkış bölgesine girdiği konum giriş bölgesine girdiği konumdan çıkarılır ve aradaki mesafe  bulunur
                mesafe=vehicles_konum[isim]-cyy

                # Araada geçen süre bu id için ilk defa hesaplandıysa buraya girer 
                if isim not in vehicles_elapsed_time:
                    # hesaplanan süre, süreler için oluşturulan sözlüğe eklenir.
                    vehicles_elapsed_time[isim] = elapsed_time

                # Daha öneceki Frame'lerde bu için  süre hesaplandıysa buraya girer.
                # bunu yapma amacımız her bir araba sadece bir kere süre hesaplamak.
                # diğer türlü sürekli süre hesaplanır ve  hesaplanan süre sürekli artar.
                # bu da hesaplamada yanlışlığa sebep olur.
                if isim in vehicles_elapsed_time:
                    elapsed_time = vehicles_elapsed_time[isim]

                # burada hızı m/s cinsinden buluyoruz.
                a_speed_ms = distance / elapsed_time
                # hızı km/h  olacak şekilde çevirme işlemi yapıyoruz.
                a_speed_kh = a_speed_ms * 3.6
                # Orijinal videoda geçen süre ile buradaki geçen süre farklı olduğu için
                # gerçek değeri bulmak için bu işlemi uyguluyoruz.
                real_kh=a_speed_kh*30/fps
                real_kh=int(real_kh)
                # Bu araba için ilk defa hız hesabı yapıldıysa bu kısma girer böylece
                # ilk elde hız tek hız olur her bir araba için
                if isim not in vehicles_speed:
                    # Burada her bir araba için bölgeler arası piksel uzaklığı farlı olabileceği için 
                    # bu işlemi uyguluıyoruz
                    real_kh=real_kh*mesafe/uzunluk
                    real_kh=int(real_kh)
                    vehicles_speed[isim]=real_kh
                # ekrana yazdırmak için str formatına çeviriyoruz.
                sag_hiz=str(vehicles_speed[isim])+' km/h'
                #Ekrana yazdırıyoruz.
                cv2.putText(kare, sag_hiz,(x, y), font, 1.7, (0,150,255), 3)
                
        ##################################################################################################### 
        # buradaki işlemleri yolun sol kısmı için yapıyrouz. Fakat mantık sağ kısım ile aynı
        inside_region3=cv2.pointPolygonTest(region3,(cx,cy),False)
        if inside_region3>0:
            vehicles_entering2[isim] = time.time()
            
            vehicles_konum2[isim]=cyy2
       
        
        
        
        if isim in vehicles_entering2:
            inside_region4=cv2.pointPolygonTest(region4,(cx,cy),False)
            
            if inside_region4>0:
                elapsed_time2 = time.time() - vehicles_entering2[isim]
                mesafe2=vehicles_konum2[isim]-cyy2
                # giriş bölgesindeki konumdan çıkış bölgesindeki konumu çıkarıyoruz.
                # Fakat yolun solunda bu işlem negatif bir sonuç verdiği için 
                # bu sonucun eksilisini bulup pozitif bir sonuç buluyrouz
                mesafe2=-mesafe2

                
                if isim not in vehicles_elapsed_time2:
                    vehicles_elapsed_time2[isim] = elapsed_time2

                
                if isim in vehicles_elapsed_time2:
                    elapsed_time2 = vehicles_elapsed_time2[isim]

          
                a_speed_ms2 = distance / elapsed_time2
                a_speed_kh2 = a_speed_ms2 * 3.6
                real_kh2=a_speed_kh2*30/fps
                real_kh2=int(real_kh2)
                if isim not in vehicles_speed2:
                    real_kh2=real_kh2*mesafe2/uzunluk2
                    real_kh2=int(real_kh2)
                    vehicles_speed2[isim]=real_kh2
                
                sol_hiz=str(vehicles_speed2[isim])+' km/h'
                cv2.putText(kare, sol_hiz,(x, y), font, 1.7, (0,255,255), 3)
                
    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
kamera.release()
cv2.destroyAllWindows()